In [ ]:
# 0. install & import & connect google drive

!pip install requests beautifulsoup4 pandas lxml

import requests, re, time, os
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 1. 사이트 검색연결

today = datetime.now().strftime("%y.%m.%d")
scientific_name = "종명작성필요" # <- 종명작성필요 에 학명 작성

base_url = "https://www.mbris.kr/pub/parcel/search/parcelSearch.do"
detail_url = "https://www.mbris.kr/pub/parcel/search/pop/openMaresDetail.do"
id_lookup_url = "https://www.mbris.kr/pub/parcel/search/searchMaresId.do"

session = requests.Session()
headers = {
    "Referer": "https://www.mbris.kr/",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
}
session.headers.update(headers)

In [ ]:
# 2. 목록에서 자원번호, maresId/Typ 추출

entries = []
page = 1
print("📌 검색 시작:", scientific_name)

while True:
    params = {"pageIndex": page, "avlParc": "all", "searchWrd": scientific_name}
    resp = session.get(base_url, params=params)
    soup = BeautifulSoup(resp.text, "html.parser")
    rows = soup.select("table tbody tr")

    if not rows:
        print(f"🔚 페이지 {page}에서 종료")
        break

    prev = len(entries)
    for row in rows:
        tds = row.find_all("td")
        if len(tds) < 6:
            continue

        res_no = tds[0].get_text(strip=True)
        area = tds[4].get_text(strip=True)
        date = tds[5].get_text(strip=True)

        onclick = ""
        view_link = row.find("a", onclick=True)
        if view_link:
            onclick = view_link["onclick"]

        mares_id, mares_typ = "", ""

        # onclick 안에 인자가 있으면 우선 사용
        args = re.findall(r"'([^']*)'", onclick)
        if len(args) == 2:
            mares_id, mares_typ = args[0].strip(), args[1].strip()
        elif len(args) == 1:
            mares_id, mares_typ = args[0].strip(), "SPCMEN"

        # onclick에서 못 찾으면 자원번호 패턴 기반으로 추정 + 별도 조회 시도
        if not mares_id:
            if "TI" in res_no:
                mares_typ = "TISSUE"
            elif "PI" in res_no:
                mares_typ = "SPCMEN"
            elif "DNA" in res_no:
                mares_typ = "DNA"
            elif "EX" in res_no:
                mares_typ = "EXTRACT"
            else:
                mares_typ = "SPCMEN"

            # 백엔드 요청 MRS 코드 조회
            try:
                q = {"searchWrd": res_no}
                r2 = session.get(id_lookup_url, params=q)
                if r2.status_code == 200 and "MRS" in r2.text:
                    m = re.search(r"(MRS\d+)", r2.text)
                    if m:
                        mares_id = m.group(1)
            except Exception:
                pass

        entries.append({
            "res_no": res_no,
            "collect_area": area,
            "collect_date": date,
            "mares_id": mares_id,
            "mares_typ": mares_typ,
        })

    print(f"📍 {page} 페이지 완료 — 누적 {len(entries)}건")

    if len(entries) == prev:
        break

    page += 1
    time.sleep(0.3)

print("🎉 리스트 수집 완료:", len(entries), "건")

📌 검색 시작: Conus lischkeanus
📍 1 페이지 완료 — 누적 10건
📍 2 페이지 완료 — 누적 20건
📍 3 페이지 완료 — 누적 28건
🔚 페이지 4에서 종료
🎉 리스트 수집 완료: 28 건


In [ ]:
# 3. 위경도 분리 - 이건 본인 편의상 넣음

def split_dms(v):
    m = re.match(r".*?(\d+)°(\d+)'([\d\.]+)", v)
    if m:
        return m.group(1), m.group(2), m.group(3)
    return "", "", ""

In [ ]:
# 4. 상세정보 수집 (채집일자 = YYYY / MM / DD만 저장)

data_list = []

for e in entries:
    rn = e["res_no"]
    mares_id = e["mares_id"]
    mares_typ = e["mares_typ"]

    if not mares_id:
        print(f"⚠️ maresId 없음 — 스킵: {rn}")
        continue

    r = session.post(detail_url, data={"maresId": mares_id, "maresTyp": mares_typ})
    if r.status_code != 200:
        print(f"❌ {rn} → status={r.status_code}")
        continue

    soup = BeautifulSoup(r.text, "html.parser")

    def tx(css_selector: str) -> str:
        tag = soup.select_one(css_selector)
        return tag.get_text(" ", strip=True) if tag else ""

    sci = tx("td[data-cell-header='학명']") or scientific_name
    nat = tx("td[data-cell-header='국명']")
    area = tx("td[data-cell-header='채집지']") or e["collect_area"]
    coord = tx("td[data-cell-header='위경도']")
    date = tx("td[data-cell-header='채집일자']") or e["collect_date"]
    collector = tx("td[data-cell-header='채집자']")

    # 보존 관련 정보들
    preservative = tx("td[data-cell-header='보존액']") or tx("td[data-cell-header='최종 보존액']")
    sample_type = tx("td[data-cell-header='시료유형']") or tx("td[data-cell-header='표본유형']") or mares_typ
    method = tx("td[data-cell-header='보존방법']")
    part = tx("td[data-cell-header='조직부위']")

    # 채집일자 → YYYY, MM, DD 분리
    date_raw = date.strip() if date else ""
    date_only = date_raw.split()[0] if date_raw else ""   # '2016-03-14 00:00:00.0' → '2016-03-14'
    year = month = day = ""
    m_date = re.match(r"(\d{4})-(\d{2})-(\d{2})", date_only)
    if m_date:
        year, month, day = m_date.group(1), m_date.group(2), m_date.group(3)

    # 위경도 정리
    coord = coord.replace("\n", " ").replace("\r", " ")
    lat_d = lat_m = lat_s = lon_d = lon_m = lon_s = ""
    if coord:
        coord_c = re.sub(r"\s+", " ", coord)
        lat = re.search(r"위도\s*[:：]?\s*([\d°'\.N]+)", coord_c)
        lon = re.search(r"경도\s*[:：]?\s*([\d°'\.E]+)", coord_c)
        if lat and lon:
            lat_d, lat_m, lat_s = split_dms(lat.group(1))
            lon_d, lon_m, lon_s = split_dms(lon.group(1))

    # 🔹 date_only(YYYY-MM-DD)는 저장하지 않고, YYYY/MM/DD만 저장
    data_list.append([
        rn,
        lat_d, lat_m, lat_s,
        lon_d, lon_m, lon_s,
        year, month, day,
        sci, area,
        collector, sample_type
    ])

    print(f"✅ {rn} ({mares_typ}) 완료")
    time.sleep(0.2)

print("✅ 상세정보 수집 완료 — 총", len(data_list), "건")


✅ MABIK MO00030347 (SPCMEN) 완료
✅ MABIK MO00030348 (SPCMEN) 완료
✅ MABIK MO00030357 (SPCMEN) 완료
✅ MABIK MO00035936 (SPCMEN) 완료
✅ MABIK GR00002136 (DNA) 완료
✅ MABIK GR00002137 (DNA) 완료
✅ MABIK GR00002138 (DNA) 완료
✅ MABIK MO00176468 (SPCMEN) 완료
✅ MABIK TI00003026 (TISSUE) 완료
✅ MABIK MO00160367 (SPCMEN) 완료
✅ MABIK MO00173104 (SPCMEN) 완료
✅ MABIK MO00173105 (SPCMEN) 완료
✅ MABIK MO00179519 (SPCMEN) 완료
✅ MABIK MO00179520 (SPCMEN) 완료
✅ MABIK MO00179521 (SPCMEN) 완료
✅ MABIK MO00179522 (SPCMEN) 완료
✅ MABIK MO00181303 (SPCMEN) 완료
✅ MABIK MO00182165 (SPCMEN) 완료
✅ MABIK MO00184689 (SPCMEN) 완료
✅ MABIK MO00188774 (SPCMEN) 완료
✅ MABIK MO00191213 (SPCMEN) 완료
✅ MABIK MO00191461 (SPCMEN) 완료
✅ MABIK MO00189820 (SPCMEN) 완료
✅ MABIK MO00189821 (SPCMEN) 완료
✅ MABIK MO00189822 (SPCMEN) 완료
✅ 4017 (SPCMEN) 완료
✅ 3074 (DNA) 완료
✅ 3073 (DNA) 완료
✅ 상세정보 수집 완료 — 총 28 건


In [ ]:
# 5. CSV 저장

df = pd.DataFrame(data_list, columns=[
    "자원번호",
    "위도(도)", "위도(분)", "위도(초)",
    "경도(도)", "경도(분)", "경도(초)",
    "채집일자(YYYY)", "채집일자(MM)", "채집일자(DD)",
    "학명", "채집지",
    "채집자", "시료/표본유형"
])

save_dir = "/content/drive/MyDrive/MBRIS"
os.makedirs(save_dir, exist_ok=True)

filename = f"{scientific_name.replace(' ', '_')}_{today}_all.csv"
full_path = os.path.join(save_dir, filename)

df.to_csv(full_path, index=False, encoding="utf-8-sig")
print("📦 CSV 저장 완료:", full_path)

df.head()

📦 CSV 저장 완료: /content/drive/MyDrive/MBRIS/Conus_lischkeanus_25.12.11_all.csv


,자원번호,위도(도),위도(분),위도(초),경도(도),경도(분),경도(초),채집일자(YYYY),채집일자(MM),채집일자(DD),학명,채집지,채집자,시료/표본유형
0,MABIK MO00030347,33,13,39.02,126,33,47.16,2011,11,20,Conus lischkeanus,"대한민국 제주근해 Seogwipo-si, Jeju-do, Korea",한성수,액침표본
1,MABIK MO00030348,33,13,39.02,126,33,47.16,2011,11,20,Conus lischkeanus,"대한민국 제주근해 Seogwipo-si, Jeju-do, Korea",한성수,액침표본
2,MABIK MO00030357,33,13,39.02,126,33,47.16,2011,11,20,Conus lischkeanus,"대한민국 제주근해 Seogwipo-si, Jeju-do, Korea",한성수,액침표본
3,MABIK MO00035936,33,13,39.02,126,33,47.16,2012,01,18,Conus lischkeanus,"대한민국 제주근해 Seogwipo-si, Jeju-do, Korea",한성수,액침표본
4,MABIK GR00002136,33,13,39.02,126,33,47.16,2012,01,18,Conus lischkeanus,"대한민국 제주근해 Seogwipo-si, Jeju-do, Korea",한성수,DNA


In [ ]:
# 6. CSV 위경도 중복 표시

import glob
import os

save_dir = "/content/drive/MyDrive/MBRIS"
csv_files = glob.glob(os.path.join(save_dir, "*.csv"))

if not csv_files:
    raise FileNotFoundError("MBRIS 폴더에 CSV 파일이 없습니다.")

latest_file = max(csv_files, key=os.path.getctime)
print("불러온 파일:", latest_file)

df = pd.read_csv(latest_file)


#위경도중복
df["중복여부"] = df.duplicated(
    subset=[
        "위도(도)", "위도(분)", "위도(초)",
        "경도(도)", "경도(분)", "경도(초)"
    ],
    keep="first"
)
df["중복여부"] = df["중복여부"].apply(lambda x: "중복됨" if x else "")

#재가공본
save_name = latest_file.replace(".csv", "_dedup.csv")
df.to_csv(save_name, index=False, encoding="utf-8-sig")

print("저장 완료:", save_name)

불러온 파일: /content/drive/MyDrive/MBRIS/Conus_lischkeanus_25.12.11_all.csv
저장 완료: /content/drive/MyDrive/MBRIS/Conus_lischkeanus_25.12.11_all_dedup.csv
